In [14]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, time ,date ,timedelta
today = date.today().strftime("%Y%m%d")

In [3]:
import gc

### Preliminars function and import

In [4]:
def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [5]:
df_articles = pd.read_csv('./Vertbaudet/France/src/Original/Articles/Export_FR_Articles_Julie.csv',
                                     usecols = ["DEPARTEMENT_NIV2","CODSAI","REFSTK"])

df_codsoc = pd.read_csv('./Vertbaudet/France/src/20210921_CODSOC.csv')

df_client_dept = pd.read_csv('./20210923_Export_tab_FR/20210921_INDIVIDUS.tab', sep = '\t',
                 usecols = ['IDIND', 'CODPOST', "DATPREMCDEWEB"])

clients_mag = pd.read_csv("./Vertbaudet/France/src/20211004_UNIQUE_IDIND_TICKET.csv",sep=",")

df_client = pd.read_csv('./Vertbaudet/France/Output_old/iris_med_rev_clients.csv', usecols = ['IDIND', 'Mediane'])

df_IRIS_by_dep = pd.read_excel('./Vertbaudet/France/INSEE_data/INSEE_IRIS_by_dep.xlsx', usecols = ['CODGEO','MED18'])

C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
gc.collect()

258

# Import

In [7]:
gc.collect()
df = pd.DataFrame()
for year in range(2017,2022):
    print(year)
    gc.collect()
    df_tmp = pd.read_csv('./Vertbaudet/France/src/Original/Lignes_commandes/Export_FR_LIGNECOMMANDE_'+str(year)+'.csv', 
                              encoding = 'unicode_escape', 
                              usecols = ['IDIND', 'IDFOYER', 'DN', 'NUMCDE', 'DBI', 'DATCDE'])

    ### Merge with article table to get DEPARTEMENT_NIV2
    #df_tmp = df_tmp.merge(df_articles, how = 'left', on = ["CODSAI","REFSTK"])
    
    ### Replace $null$
    df_tmp = df_tmp.replace({'$null$': np.nan})
    ### Clean IDIND index 
    df_tmp.IDIND = df_tmp.IDIND.astype(float)
    df_tmp.IDFOYER = df_tmp.IDFOYER.astype(float)
    
    ### Filter mag clients
    df_tmp = df_tmp[~df_tmp.IDIND.isin(clients_mag.IDIND)]

    ### Filter non-french command with CODSOC
    df_tmp = df_tmp.merge(df_codsoc, how = 'left', on = 'NUMCDE')
    df_tmp = df_tmp[df_tmp.CODSOC==0]

    #### Merge to get Mediane rev
    df_tmp = df_tmp.merge(df_client, how = 'left', on = 'IDIND')

    ### add departments and cohort
    df_tmp = df_tmp.merge(df_client_dept.replace({'$null$': np.nan}), how = 'left', on = 'IDIND')
    df_tmp = df_tmp[df_tmp.DATPREMCDEWEB != '$null$']
    df_tmp['DEPARTEMENT'] = df_tmp['CODPOST'].apply(lambda x : str(x)[:2])
    df_tmp['COHORT'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"]).dt.year
    
    ### filter cohort
    df_tmp = df_tmp[(df_tmp['COHORT'] >= 2017) & (df_tmp['COHORT'] < 2021)]
    
    ### filter on DN <= DBI
    #df_tmp = df_tmp[df_tmp.DN <= df_tmp.DBI]
    #df_tmp = df_tmp[df_tmp.DN >= 0]

    df = pd.concat([df,df_tmp])
  
    del df_tmp

2017


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2018


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2019


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2020


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2021


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df.DN.sum()

254120319.3899999

In [9]:
df['DATPREMCDEWEB'] = pd.to_datetime(df['DATPREMCDEWEB'])

In [10]:
# Exclude B2B clients
df_b2b = pd.read_csv('./Vertbaudet/France/src/20211012_TB2B.csv')
df_b2b[['IDIND', 'TB2B']] = df_b2b[['IDIND', 'TB2B']].astype(float)
df = df.merge(df_b2b, how = 'left', on = 'IDIND')
df = df[df['TB2B']==0]
df = df.drop(columns = ['TB2B'])
del df_b2b

In [11]:
# Add DAT_12 column
df['DAT_12'] = df['DATPREMCDEWEB'] + timedelta(days=365)
df['DAT_24'] = df['DATPREMCDEWEB'] + timedelta(days=730)

In [18]:
### Add LTV column
df_ltv_12 = df[df['DATCDE']<=df['DAT_12']][['IDIND', 'DN']]
df_ltv_12 = df_ltv_12.groupby('IDIND').agg({'DN': sum})
df_ltv_12 = df_ltv_12.rename(columns = {'DN': 'LTV_12'})
df_ltv_12['LTV_12'].mean()
df = df.merge(df_ltv_12, how = 'left', on = 'IDIND')

del df_ltv_12

In [24]:
### Add LTV column
df_ltv_24 = df[df['DATCDE']<=df['DAT_24']][['IDIND', 'DN']]
df_ltv_24 = df_ltv_24.groupby('IDIND').agg({'DN': sum})
df_ltv_24 = df_ltv_24.rename(columns = {'DN': 'LTV_24'})
df_ltv_24['LTV_24'].mean()
df = df.merge(df_ltv_24, how = 'left', on = 'IDIND')

del df_ltv_24

In [15]:
df['SENIORITY'] = (datetime.today() - df['DATPREMCDEWEB']).dt.days

- Create output df and to_excel

In [19]:
df.columns

Index(['IDIND', 'IDFOYER', 'DATCDE', 'NUMCDE', 'DBI', 'DN', 'CODSOC',
       'Mediane', 'DATPREMCDEWEB', 'CODPOST', 'DEPARTEMENT', 'COHORT',
       'DAT_12', 'DAT_24', 'SENIORITY', 'LTV_12'],
      dtype='object')

In [20]:
df_LTV_12_by_dept = df[df['SENIORITY']>365].drop_duplicates('IDIND').groupby('DEPARTEMENT').agg({'LTV_12': 'mean'})
df_LTV_12_by_dept.to_excel('./Vertbaudet/France/Output/'+today+'_ltv_12_by_dep.xlsx')

In [25]:
df[df['SENIORITY']>365].drop_duplicates('IDIND')['LTV_12'].mean()

120.59403348484754

 ### CSP Slices 

In [16]:
def csp_map(x):
    if x < 15000:
        return '<15k'
    elif x < 20000:
        return '15-20k'
    elif x < 25000:
        return '20-25k'
    elif x < 30000:
        return '25-30k'
    elif x > 30000:
        return '>30k'
df['CSP'] = df['Mediane'].apply(csp_map)

In [17]:
df_csp_by_cohort = df.groupby(['COHORT', 'CSP']).agg({'IDIND': pd.Series.nunique})

In [18]:
df_csp_by_cohort.unstack().to_excel('./Vertbaudet/France/Output/'+today+'_csp_by_cohort.xlsx')

In [22]:
def csp_map_detail(x):
    if x < 15000:
        return '<15k'
    elif x < 20000:
        return '15-20k'
    elif x < 21000:
        return '20-21k'
    elif x < 22000:
        return '21-22k'
    elif x < 23000:
        return '22-23k'
    elif x < 24000:
        return '23-24k'
    elif x < 25000:
        return '24-25k'
    elif x < 30000:
        return '25-30k'
    elif x > 30000:
        return '>30k'
df['CSP_det'] = df['Mediane'].apply(csp_map_detail)

In [23]:
df_csp_det_by_cohort = df.groupby(['COHORT', 'CSP_det']).agg({'IDIND': pd.Series.nunique})

In [24]:
df_csp_det_by_cohort.unstack().to_excel('./Vertbaudet/France/Output/'+today+'_csp_det_by_cohort.xlsx')

In [28]:
df.drop_duplicates('IDIND').groupby('COHORT').agg({'Mediane': 'mean'})

,Mediane
COHORT,
2017.0,22892.874486
2018.0,22812.551147
2019.0,22755.562354
2020.0,22780.712051
